# Fashion MNIST Classification

## Importing the Fashion MNIST Dataset

In [ ]:
# import statements

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras_tuner as kt

print(tf.__version__)

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Exploring the Data

In [ ]:
train_images.shape

In [ ]:
len(train_labels)

In [ ]:
test_images.shape

In [ ]:
len(test_images)

## Preprocessing

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
# Scaling all the values in the bitmap images to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Plotting 25 images and their label
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Creating the Model

### Defining the Model

In [ ]:
def model_builder(hp):
    # Create the model
    model = tf.keras.Sequential()
    
    # Input Layer
    model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
    
    hp_units = [0] * 7

    # Tuning the number of nodes in the first layer
    for i in range(hp.Int("num_layers", 2, 6)):
        hp_units[i] = hp.Int("units_" + str(i), min_value=32, max_value=1024, step=64)
        #hp_reg_rate = hp.Choice("regularization_rate", values=[0.1])
        model.add(tf.keras.layers.Dense(units=hp_units[i], activation="relu"))
    
    hp_drop_rate = hp.Choice("dropout_rate", values=[0.2, 0.3, 0.4, 0.5])
    model.add(tf.keras.layers.Dropout(0.30))
    model.add(tf.keras.layers.Dense(10))    

    # Tune the learning rate from 0.01, 0.001, and 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    # Compiling the neural network
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    return model

### Perform hypertuning

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='try3',
                     project_name='optimizingClothingClassification')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(train_images, train_labels, epochs=50, validation_split=0.2, callbacks=[stop_early], batch_size=1200)

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get("num_layers"))
print(best_hps.get("units_0"))
print(best_hps.get("units_1"))
print(best_hps.get("units_2"))
print(best_hps.get("learning_rate"))

In [ ]:
model.summary()

### Train the Model

In [ ]:
# Training the model
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history["val_accuracy"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print("Best epoch: %d" % (best_epoch,))


In [ ]:
# Retrain with best epoch
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain
hypermodel.fit(train_images, train_labels, epochs=best_epoch, validation_split=0.2)

In [ ]:
# Accuracy on Test Set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("\nTest accuracy:", test_acc)

In [ ]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model(test_images)
predictions[0]
np.argmax(predictions[0])


In [ ]:
test_labels[0]

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    true_label, img = true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'
    
    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
    100*np.max(predictions_array), class_names[true_label]),color=color)

def plot_value_array(i, predictions_array, true_label):
    true_label = true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0,1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color("red")
    thisplot[true_label].set_color("blue")


In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i], test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i], test_labels)
plt.show()

In [ ]:
# Visualization of the results
num_rows = 5
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

In [ ]:
# Testing on a single image
img = test_images[1]
print(img.shape)
img = np.expand_dims(img,0)
print(img.shape)

predictions_single = probability_model.predict(img)
print(predictions_single)
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
plt.show()
print(class_names[np.argmax(predictions_single[0])])